# 1.7 字典排序
* 问题：创建一个字典，并且在迭代后者序列化字典时能够 控制元素的顺序
* 方案：可以使用collections模块中的OrderedDict类。在迭代操作时他会保持元素被插入时的顺序

In [1]:
from collections import OrderedDict
d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

for key in d:
    print(key,': ',d[key])

foo :  1
bar :  2
spam :  3
grok :  4


* 当构建一个将来需要序列化或者编码成其他格式的映射时，OrderedDict会很有用
* 选要注意的是：OrderedDict对象的大小是一个普通字典的两倍，因为其内部维护着另外一个链表。当需要读取大量数据到OrderedDict字典时就要多考虑一下

In [7]:
import json
with open('data_file/test1.json','w') as f:
    json.dump(d,f)
with open('data_file/test1.json','r') as f:
    d = json.load(f)
    print(d)

{'foo': 1, 'bar': 2, 'spam': 3, 'grok': 4}


# 字典的运算
* 问题：怎样在数据字典中执行计算操作
* 方案：考虑使用zip() 函数,先将键值对反转过来。
* 如果直接在字典上之执行普通的数学运算，会发现它只作用于键

In [14]:
price = {
    'ACME':45.23,
    'AAPL':456.12,
    'IBM':132.21,
    'HPQ':23.2,
    'FB':10.75
}
min_price = min(zip(price.values(),price.keys()))
max_price = max(zip(price.values(),price.keys()))
print(min_price,'\n',max_price)

(10.75, 'FB') 
 (456.12, 'AAPL')


* 注意：zip()函数创建的是一个只能访问一次的迭代器，下面的方式会报错

In [15]:
price_name = zip(price.values(),price.keys())
print(min(price_name))
print(max(price_name))

(10.75, 'FB')


ValueError: max() arg is an empty sequence

In [16]:
max(price)

'IBM'

In [17]:
price.values()

dict_values([45.23, 456.12, 132.21, 23.2, 10.75])

* 可以在min()和max() 函数中提供key

In [21]:
min(price,key = lambda k:price[k])

'FB'

In [22]:
max(price,key = lambda k:price[k])

'AAPL'

* 但是有时候如果几个键对应的值相同，在对其进行键值反转求最小值时，会根据键的大小返回结果

In [25]:
price1 = {
    'A':1,
    'B':2,
    'C':1,
    'D':3
}
min_price = min(zip(price1.values(),price1.keys()))

In [26]:
min_price

(1, 'A')

In [27]:
min([(1,'a'),(2,'b'),(3,'a')])

(1, 'a')

* zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的对象，这样做的好处是节约了不少的内存。我们可以使用 list() 转换来输出列表。如果各个迭代器的元素个数不一致，则返回列表长度与最短的对象相同，利用 * 号操作符，可以将元组解压为列表。

In [34]:
a = [1,2,3]
b = [4,5,6]
c = ['a','b','c','d']
zipped1 = zip(a,b)
zipped2 = zip(a,c)

In [35]:
print(list(zipped1))
print(list(zipped2))

[(1, 4), (2, 5), (3, 6)]
[(1, 'a'), (2, 'b'), (3, 'c')]


# 1.9查找字典的相同点
* 问题：怎样在字典中查找相同的点（比如相同的 键 或者相同的 值）
* 方案：可以简单的在两个字典的keys()或者items()方法的返回结果上执行   集合   操作
* keys()返回一个一个字典的所有键的视图对象。items()返回一个（键，值）对的元素视图对象

In [45]:
#自己想的普通方法
a = {
    'x':1,
    'y':2,
    'z':3
}
b = {
    'w':10,
    'x':11,
    'y':2
}

In [46]:
for key in a.keys():
    if key in b.keys():
        print(key)

x
y


* 利用集合操作,但是不可以对字典的值进行操作。如果非要对值进行集合操作，可以先将其转换为set。

In [48]:
a.keys() & b.keys()

{'x', 'y'}

In [49]:
a.keys() - b.keys()

{'z'}

In [50]:
a.items() & b.items()

{('y', 2)}

In [53]:
a.values() & b.values()

TypeError: unsupported operand type(s) for &: 'dict_values' and 'dict_values'

* 上述操作也可以对字典进行过滤。比如删除字典指定的键

In [54]:
c = {key:a[key] for key in a.keys()-{'z','w'}}

In [55]:
c

{'x': 1, 'y': 2}

# 1.10删除序列相同元素并保持顺序
* 问题：怎样在一个序列上面保持元素顺序的同时消除重复元素
* 方案：如果对象是hashable，则可以利用集合或者生成器解决问题

In [57]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [59]:
a = [1,2,3,4,5,2,1,6,7,3,1]
list(dedupe(a))

[1, 2, 3, 4, 5, 6, 7]

* 上述代码并不适用于dict对象，因为dict不是可哈希的。现对其修改。

In [62]:
def dedupe(items,key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

In [63]:
a = [{'x':1,'y':2},{'x':1,'y':3},{'x':1,'y':2},{'x':3,'y':4}]
list(dedupe(a,key=lambda d:(d['x'],d['y'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 3, 'y': 4}]

In [65]:
list(dedupe(a,key=lambda d:d['x']))

[{'x': 1, 'y': 2}, {'x': 3, 'y': 4}]

# 1.11命名切片
* 问题：有一段代码要从记录中取出固定位置的数据字段
* 方案：可以使用直接切片，但更好的是使用命名切片。slice()函数

In [70]:
record = '............100..........121.2........'
cost = int(record[12:15])*float(record[25:29])
cost

12100.0

In [75]:
SHARES = slice(12,15)
PRICE = slice(25,29)
cost = int(record[SHARES])*float(record[PRICE])
cost

12100.0

* 内置函数slice()可以使用在任何可以切片的地方，使用命名切片的原因主要是方便日后的维护

In [80]:
items = [0,1,2,3,4,5,6]
a = slice(2,5)

In [81]:
items[2:5]

[2, 3, 4]

In [82]:
items[a]

[2, 3, 4]

In [84]:
items[2:4] = [10,11]
items

[0, 1, 10, 11, 4, 5, 6]

In [85]:
items[a] = [14,15,16]

In [86]:
items

[0, 1, 14, 15, 16, 5, 6]

* 另外slice()函数可以指定步长。slice(n,m,s)
* slice对象拥有属性start,stop，step

In [87]:
a = slice(1,20,2)

In [88]:
a.start

1

In [89]:
a.stop

20

In [90]:
a.step

2

* 通过调用切片的 indices(size) 方法将它映射到一个确定大小的序列上，这个方法返回一个三元组 (start, stop, step) ，所有值都会被合适的缩小以满足边界限制，从而使用的时候避免出现 IndexError 异常。

In [95]:
s = "helloworld"
a.indices(len(s))

(1, 10, 2)

In [96]:
for i in range(*a.indices(len(s))):
    print(s[i])

e
l
w
r
d


# 查找序列中出现最多的元素
* 问题：怎样查找出序列中出现次数最多的元素
* 方案：使用collections.Counter类,它拥有一个most_common()方法可以直接给出答案
* 作为输入， Counter 对象可以接受任意的由可哈希（hashable）元素构成的序列对象。在底层实现上，一个 Counter 对象就是一个字典，将元素映射到它出现的次数上。

In [104]:
words = [
'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
'my', 'eyes', "you're", 'under'
]
morewords = ['why','are','you','not','looking','in','my','eyes']
from collections import Counter
word_counts = Counter(words)
top_three = word_counts.most_common(3)
top_three

[('eyes', 8), ('the', 5), ('look', 4)]

In [105]:
#实际上是一个字典
word_counts

Counter({'look': 4,
         'into': 3,
         'my': 3,
         'eyes': 8,
         'the': 5,
         'not': 1,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

* Counter实例有一个特性：他们可以很容易的跟数学运算操作符相结合

In [106]:
a = Counter(words)
b = Counter(morewords)

In [109]:
a+b

Counter({'look': 4,
         'into': 3,
         'my': 4,
         'eyes': 9,
         'the': 5,
         'not': 2,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1,
         'why': 1,
         'are': 1,
         'you': 1,
         'looking': 1,
         'in': 1})

In [110]:
a - b

Counter({'look': 4,
         'into': 3,
         'my': 2,
         'eyes': 7,
         'the': 5,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

# 1.13按制定的关键字排序字典
* 问题：根据某个或者某几个字段排序一个列表
* 方案：通过使用 operator 模块的 itemgetter 函数，可以非常容易的排序这样的数据结构。

In [113]:
rows = [
{'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
{'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
{'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}
]

from operator import itemgetter
rows_by_name = sorted(rows,key=itemgetter('fname'))
rows_by_uid = sorted(rows,key=itemgetter('uid'))
print(rows_by_name,'\n')
print(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}] 

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


* itemgetter()支持多个key

In [115]:
rows_by_lfname = sorted(rows,key=itemgetter('lname','fname'))
rows_by_lfname

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]

* itemgetter()可以使用lambda 表达式代替，但前者速度更快

In [117]:
rows_by_fname = sorted(rows,key=lambda r:r['fname'])
rows_by_fname

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]

In [118]:
min(rows,key=lambda r:r['fname'])

{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}

# 1.14 排序不支持原生比较的对象
* 问题：怎样排序一个不支持原生比较的对象
* 方案：使用sorted()函数，它接收一个关键字参数key.
* 方案:另外一种可以使用operator.attrgetter()

In [122]:
class User:
    def __init__(self,user_id):
        self.user_id = user_id
    
    def __repr__(self):
        return 'User({})'.format(self.user_id)
    
def sort_notcompare():
    users = [User(99),User(34),User(88)]
    print(users)
    print(sorted(users,key=lambda u:u.user_id))
    
sort_notcompare()

[User(99), User(34), User(88)]
[User(34), User(88), User(99)]


In [124]:
from operator import attrgetter
users = [User(99),User(34),User(88)]
sorted(users,key = attrgetter('user_id'))

[User(34), User(88), User(99)]